# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

## Setup: Import what you'll need to scrape the page

We'll be using Selenium for this, *not* BeautifulSoup and requests.

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for cosmetologist violations for people with the last name **Nguyen**.

In [3]:
driver =webdriver.Chrome()
driver.get('https://www.tdlr.texas.gov/cimsfo/fosearch.asp')

In [4]:
dropdown = Select(driver.find_element_by_name('pht_status'))
dropdown.select_by_visible_text('Cosmetologists')

In [5]:
text_input= driver.find_element_by_xpath('//*[@id="dat-menu"]/div/div[2]/div/div/section/div/div/table/tbody/tr/td/form/table/tbody/tr[7]/td/p/input')
text_input.send_keys('Nguyen')
 

In [6]:
search_buttom= driver.find_element_by_xpath('//*[@id="dat-menu"]/div/div[2]/div/div/section/div/div/table/tbody/tr/td/form/table/tbody/tr[18]/td/input[1]')
search_buttom.click()

## Scraping

Once you are on the results page, do this.

### Loop through each result and print the entire row

Okay wait, that's a heck of a lot. Use `[:10]` to only do the first ten (`listname[:10]` gives you the first ten).

In [11]:
table = driver.find_element_by_tag_name('tbody')
rows = table.find_elements_by_tag_name('tr')[1:10]
for row in rows:
    print(row.find_element_by_tag_name('td').text)
    


NGUYEN, TOAN HUU
City: SAN ANTONIO
County: BEXAR
Zip Code: 78217


License #(s): 780948, 1706491, 1699123

Complaint # COS20180004289
NGUYEN, HANH CONG
City: EL PASO
County: EL PASO
Zip Code: 79934


License #: 737708

Complaint # COS20180006594
NGUYEN, KHIEM VAN
City: LONGVIEW
County: GREGG
Zip Code: 75604


License #: 731665

Complaint # COS20180000257
NGUYEN, DIEP THI NGOC
City: HOUSTON
County: HARRIS
Zip Code: 77014


License #(s): 1347649, 760528

Complaint # COS20180004915
NGUYEN, LAN T-THUY
City: SAN ANTONIO
County: BEXAR
Zip Code: 78255

NGUYEN, SAMLOI
City: SAN ANTONIO
County: BEXAR
Zip Code: 78255


License #: 767339

Complaint # COS20180009255
NGUYEN, TUAN A
City: AUSTIN
County: TRAVIS
Zip Code: 78723

NGUYEN, TUAN VAN
City: ARLINGTON
County: TARRANT
Zip Code: 76011


License #: 681274

Complaint # COS20140018343
NGUYEN, THAO B
City: EULESS
County: TARRANT
Zip Code: 76039


License #(s): 721373, 1142884

Complaint # COS20180008846
NGUYEN, BETH MARIA
City: HOUSTON
County: HAR

### Loop through each result and print each person's name

You'll get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   try to do something
except:
   print("It didn't work')
```

It should help you out. If you don't want to print anything, you can type `pass` instead of the `print` statement.

**Why doesn't the first one have a name?**

In [27]:
table = driver.find_element_by_tag_name('tbody')
rows = table.find_elements_by_tag_name('tr')[1:10]
for row in rows:
    namelocations= row.find_elements_by_tag_name('td')[0]
    name= namelocations.find_elements_by_tag_name ('span')[0]
#     for namelocation in namelocations:
#         name= namelocation.find_elements_by_tag_name('span')[0]
    print(name.text)
        

NGUYEN, TOAN HUU
NGUYEN, HANH CONG
NGUYEN, KHIEM VAN
NGUYEN, DIEP THI NGOC
NGUYEN, LAN T-THUY
NGUYEN, TUAN A
NGUYEN, THAO B
NGUYEN, BETH MARIA
NGUYEN, TRUNG N


## Loop through each result, printing each violation description ("Basis for order")

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: You can get the HTML of something by doing `.get_attribute('innerHTML')` - it might help you diagnose your issue.*
> - *Tip: Or I guess you could just skip the one with the problem...

In [28]:
table = driver.find_element_by_tag_name('tbody')
rows = table.find_elements_by_tag_name('tr')[1:10]
for row in rows:
    basis= row.find_elements_by_tag_name('td')[2]
    print(basis.text)

Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day.
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day; Respondent failed to use items subject to possible cross contamination in a manner that does not contaminate the remaining product.
Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect all wax pots.
Respondent failed to disinfect tools, implements, and supplies with an EPA-registered disinfectant solution; Respondent failed to disinfect multi-use equipment, implements, and tools prior to use on each client.
Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use.
Respondent failed to clean and disinfect all wax pots; Respondent failed to disinfect tools, implements, an

## Loop through each result, printing the complaint number

- TIP: Think about the order of the elements

In [32]:
table = driver.find_element_by_tag_name('tbody')
rows = table.find_elements_by_tag_name('tr')[1:10]
for row in rows:
    namelocations= row.find_elements_by_tag_name('td')[0]
    
    complaint= namelocations.find_elements_by_tag_name ('span')[-1]
    print(complaint.text)

COS20180004289
COS20180006594
COS20180000257
COS20180004915
COS20180009255
COS20140018343
COS20180008846
COS20180000897
COS20170023893


## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

> *Tip: If you want to ask for the "next sibling," you can't use `find_next_sibling` in Selenium, you need to use `element.find_element_by_xpath("following-sibling::div")` to find the next div, or `element.find_element_by_xpath("following-sibling::*")` to find the next anything.

In [37]:
mylist=[]
table = driver.find_element_by_tag_name('tbody')
rows = table.find_elements_by_tag_name('tr')[1:10]

for row in rows:
    dic={}
    
    namelocations= row.find_elements_by_tag_name('td')[0]
    name= namelocations.find_elements_by_tag_name ('span')[0]
    dic["Person's name"]=name.text
    
    basis= row.find_elements_by_tag_name('td')[2]
    dic['Violation description']=basis.text 
    
    complaint= namelocations.find_elements_by_tag_name ('span')[-1]
    dic['Viodlation number']=complaint.text
    
    license = namelocations.find_elements_by_tag_name ('span')[-3]
    dic['License Numbers']=license.text
    
    zipcode= namelocations.find_elements_by_tag_name ('span')[6]
    dic['Zip Code']=zipcode.text
    
    county= namelocations.find_elements_by_tag_name ('span')[4]
    dic['County']=county.text
    
    city = namelocations.find_elements_by_tag_name ('span')[2]
    dic['City']=city.text
    
    mylist.append(dic)
mylist

[{"Person's name": 'NGUYEN, TOAN HUU',
  'Violation description': 'Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day.',
  'Viodlation number': 'COS20180004289',
  'License Numbers': '780948, 1706491, 1699123',
  'Zip Code': '78217',
  'County': 'BEXAR',
  'City': 'SAN ANTONIO'},
 {"Person's name": 'NGUYEN, HANH CONG',
  'Violation description': 'Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day; Respondent failed to use items subject to possible cross contamination in a manner that does not contaminate the remaining product.',
  'Viodlation number': 'COS20180006594',
  'License Numbers': '737708',
  'Zip Code': '79934',
  'County': 'EL PASO',
  'City': 'EL PASO'},
 {"Person's name": 'NGUYEN, KHIEM VAN',
  'Violation description': 'Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondent failed to clean, disinfect, and sterilize manicure and pedic

### Save that to a CSV

- Tip: You'll want to use pandas here

In [38]:
import pandas as pd
df =pd.DataFrame(mylist)
df.head()

,City,County,License Numbers,Person's name,Viodlation number,Violation description,Zip Code
0,SAN ANTONIO,BEXAR,"780948, 1706491, 1699123","NGUYEN, TOAN HUU",COS20180004289,Respondent failed to clean and sanitize whirlp...,78217
1,EL PASO,EL PASO,737708,"NGUYEN, HANH CONG",COS20180006594,Respondent failed to clean and sanitize whirlp...,79934
2,LONGVIEW,GREGG,731665,"NGUYEN, KHIEM VAN",COS20180000257,Respondent failed to follow whirlpool foot spa...,75604
3,HOUSTON,HARRIS,"1347649, 760528","NGUYEN, DIEP THI NGOC",COS20180004915,"Respondent failed to disinfect tools, implemen...",77014
4,SAN ANTONIO,BEXAR,767339,"NGUYEN, LAN T-THUY",COS20180009255,"Respondent failed to clean, disinfect, and ste...",78255


In [39]:
df.to_csv('texas_cosmetology_violations.csv',index=False)

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.